This is a assignment to deal with unhealthy conversations. The assignment description website is in
https://ml.auc-computing.nl/assignment3.html. The repository I used for the assignment is https://github.com/conversationai/unhealthy-conversations. There are some files I will use in my model: train.csv, test.csv, val.csv. 

In [296]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action="ignore")

In [297]:
data_dir = Path("corpus/")
train_data= pd.read_csv(data_dir / "train.csv")
test_data = pd.read_csv(data_dir / "test.csv")

I delete 'trusted judgements', 'unit id', 'comment' in the columns_use, and the target_column would be the 'healthy' column.

In [298]:

columns_use = ['antagonise','antagonise:confidence','condescending','condescending:confidence',
              'dismissive','dismissive:confidence','generalisation','generalisation:confidence',
              'generalisation_unfair', 'generalisation_unfair:confidence','hostile',
               'hostile:confidence', 'sarcastic','sarcastic:confidence']
target_column = ['healthy']

X_train = train_data[columns_use]

y_train = train_data[target_column]
X_test = test_data[columns_use]
y_test = test_data[target_column]


Create a function combining each sub-attributes with its confidence score, and apply the function to X_train, and X_test. The way to combine the attributes with confidence socre is find the confidence scoe when the attribute is true. Therefore, if the attribute is 1, the confidence score should be 1*score, and if the attribute is 0, the confidence score should be 1*(1-score). Combining the two calculation, the new column should be 1*score + (1-1)*(1-score) for true, and 0*score + (1-0)*(1-score). After giving a new data to the attributes, we drop the columns of attributes-level.

In [299]:
attributes = ['antagonise', 'condescending', 'dismissive', 'generalisation', 
             'generalisation_unfair', 'hostile', 'sarcastic']
confidence = ['antagonise:confidence','condescending:confidence', 
                         'dismissive:confidence', 'generalisation:confidence', 
                         'generalisation_unfair:confidence', 'hostile:confidence', 
                         'sarcastic:confidence']
    
def preprocess(X):
    for i in range(len(attributes)):
        X[attributes[i]] = X[attributes[i]] * X[confidence[i]] + (1- X[attributes[i]]) * (1-X[confidence[i]])
        
preprocess(X_train)
X_train = X_train.drop(confidence, axis=1)
preprocess(X_test)
X_test = X_test.drop(confidence, axis=1)




,antagonise,condescending,dismissive,generalisation,generalisation_unfair,hostile,sarcastic
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000,0.1959,0.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.2471,0.2471,0.2471,0.2471,0.0000,0.2471,0.0000
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3932
...,...,...,...,...,...,...,...
4420,0.4003,0.6129,0.4003,0.6129,0.6129,0.4003,0.0000
4421,0.6726,0.6726,1.0000,0.1697,0.0000,0.6726,0.3306
4422,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4423,0.4046,0.4046,0.4046,0.1974,0.0000,0.4046,0.1974
